In [1]:
from huggingface_hub import login

login()

In [2]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig

## Format dataset

In [12]:
# Load dataset
dataset = load_dataset("trl-lib/ultrafeedback_binarized")

## Select the model

We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it it compatible with DPO. You can also use the model you trained in [1_supervised_finetuning](../1_supervised_finetuning/notebooks/sft_finetuning_example.ipynb).


In [4]:
dataset[0]

{'chosen': [{'content': 'Use the pygame library to write a version of the classic game Snake, with a unique twist',
   'role': 'user'},
  {'content': "Sure, I'd be happy to help you write a version of the classic game Snake using the pygame library! Here's a basic outline of how we can approach this:\n\n1. First, we'll need to set up the game display and create a game object that we can use to handle the game's state.\n2. Next, we'll create the game's grid, which will be used to represent the game board. We'll need to define the size of the grid and the spaces within it.\n3. After that, we'll create the snake object, which will be used to represent the player's movement. We'll need to define the size of the snake and the speed at which it moves.\n4. We'll also need to create a food object, which will be used to represent the food that the player must collect to score points. We'll need to define the location of the food and the speed at which it moves.\n5. Once we have these objects se

In [5]:
# TODO: 🦁 change the model to the path or repo id of the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb)

model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float32
).to(device)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-DPO"
finetune_tags = ["smol-course", "module_1"]

## Train model with DPO

In [6]:
# Training arguments
training_args = DPOConfig(
    # Training batch size per GPU
    per_device_train_batch_size=4,
    # Number of updates steps to accumulate before performing a backward/update pass
    # Effective batch size = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps=4,
    # Saves memory by not storing activations during forward pass
    # Instead recomputes them during backward pass
    gradient_checkpointing=True,
    # Base learning rate for training
    learning_rate=5e-5,
    # Learning rate schedule - 'cosine' gradually decreases LR following cosine curve
    lr_scheduler_type="cosine",
    # Total number of training steps
    max_steps=200,
    # Disables model checkpointing during training
    save_strategy="no",
    # How often to log training metrics
    logging_steps=1,
    # Directory to save model outputs
    output_dir="smol_dpo_output",
    # Number of steps for learning rate warmup
    warmup_steps=100,
    # Use bfloat16 precision for faster training
    bf16=True,
    # Disable wandb/tensorboard logging
    report_to=None,
    # Keep all columns in dataset even if not used
    remove_unused_columns=False,
    # Enable MPS (Metal Performance Shaders) for Mac devices
    # Model ID for HuggingFace Hub uploads
    hub_model_id=finetune_name,
)

In [7]:
trainer = DPOTrainer(
    # The model to be trained
    model=model,
    # Training configuration from above
    args=training_args,
    # Dataset containing preferred/rejected response pairs
    train_dataset=dataset,
    # Tokenizer for processing inputs
    processing_class=tokenizer,
    # DPO-specific temperature parameter that controls the strength of the preference model
    # Lower values (like 0.1) make the model more conservative in following preferences
    beta=0.1,
    # Maximum length of the input prompt in tokens
    max_prompt_length=1024,
    # Maximum combined length of prompt + response in tokens
    max_length=1536,
)

c:\Users\kevor\anaconda3\envs\smol_course\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_prompt_length, max_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in DPOTrainer, please use the DPOConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\kevor\anaconda3\envs\smol_course\Lib\site-packages\trl\trainer\dpo_trainer.py:469: UserWarning: You passed `max_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
c:\Users\kevor\anaconda3\envs\smol_course\Lib\site-packages\trl\trainer\dpo_trainer.py:475: UserWarning: You passed `max_prompt_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# Train the model
trainer.train()



  0%|          | 0/200 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 0.6931, 'grad_norm': 80.11817932128906, 'learning_rate': 5.000000000000001e-07, 'rewards/chosen': 0.0, 'rewards/rejected': 0.0, 'rewards/accuracies': 0.0, 'rewards/margins': 0.0, 'logps/chosen': -453.22967529296875, 'logps/rejected': -332.1123352050781, 'logits/chosen': 4.746685028076172, 'logits/rejected': 4.014349937438965, 'epoch': 0.0}
{'loss': 0.6931, 'grad_norm': 84.1331558227539, 'learning_rate': 1.0000000000000002e-06, 'rewards/chosen': 0.0, 'rewards/rejected': 0.0, 'rewards/accuracies': 0.0, 'rewards/margins': 0.0, 'logps/chosen': -448.68798828125, 'logps/rejected': -387.6080322265625, 'logits/chosen': 4.019976615905762, 'logits/rejected': 5.054856300354004, 'epoch': 0.0}
{'loss': 0.7038, 'grad_norm': 93.25899505615234, 'learning_rate': 1.5e-06, 'rewards/chosen': -0.02288069762289524, 'rewards/rejected': -0.00739463372156024, 'rewards/accuracies': 0.4375, 'rewards/margins': -0.015486061573028564, 'logps/chosen': -632.8018798828125, 'logps/rejected': -477.3955993652344

NameError: name 'x' is not defined

In [10]:
# Save the model
trainer.save_model(f"./{finetune_name}")

# Save to the huggingface hub if login (HF_TOKEN is set)
if os.getenv("HF_TOKEN"):
    trainer.push_to_hub(tags=finetune_tags)

In [11]:
trainer.push_to_hub(tags=finetune_tags)

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/6.07k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/herooooooooo/SmolLM2-FT-DPO/commit/e9f531fd0d7fe79cae6dafc28bf80c6295e95ff2', commit_message='End of training', commit_description='', oid='e9f531fd0d7fe79cae6dafc28bf80c6295e95ff2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/herooooooooo/SmolLM2-FT-DPO', endpoint='https://huggingface.co', repo_type='model', repo_id='herooooooooo/SmolLM2-FT-DPO'), pr_revision=None, pr_num=None)